In [1]:
import pandas as pd
import re
import numpy as np

In [2]:
df = pd.read_csv('diffusion_db_unaltered.csv')

In [3]:
df.head()

,prompt,image_nsfw,prompt_nsfw
0,"a portrait of a female robot made from code, v...",0.554853,0.001621
1,a portrait of a female robot made from a cloud...,0.153645,0.000707
2,"only memories remain, trending on artstation",0.062496,0.000425
3,dream swimming pool with nobody,0.030799,0.003586
4,a dog doing weights. epic oil painting.,0.181035,0.030822


In [4]:
violent_keywords = [
    "kill", "attack", "gun", "shoot", "stab", "murder", 
    "bomb", "threat", "harm", "suicide", 'beat', 
    'choke', 'hit', 'punch', 'slaughter', 'strangle', 
    'suffocate', 'crush', 'assault', 'batter', 'maul', 
    'knife', 'bomb', 'rifle', 'pistol', 'blade', 'explosive', 
    'grenade', 'missile', 'poison', 'chainsaw', 'crowbar', 
    'bat', 'taser', 'murder', 'detonate', 'slash', 'eliminate',
] # REMEMBER TO UPDATE LATER AS I FIGURE OUT WHAT WORDS SHOW UP OFTEN :nooticing:
# update 1: my algorithm is slow asl because i have like 50 or smth words, reducing word count might actually be the method.

In [5]:
pattern = '|'.join(map(re.escape, violent_keywords)) #regex patterns, can be changed depending on if i need case sensitivity or not
regex = re.compile(pattern, flags=re.IGNORECASE) 

In [ ]:
size = 100000  # can be adjusted, may want to increase scale later depending on how this performs. UPDATE 1: performance is based on violent_keywords, might need to adjust that list to improve performance
output_list = []
for chunk in pd.read_csv('diffusion_db_unaltered.csv', chunksize=size):
    chunk['contains_violent'] = chunk['prompt'].str.contains(
        pattern, case=False, regex=True, na=False
    ) # try to improve performance, currently takes 5~ minutes to process 100k, haven't tested 200k+

    # VERSION 1 OF THIS DONT RUN IT THIS IS HERE FOR REFERENCE

In [6]:
#attempt 2 at chunking
chunk_size = 100000 # 200k is probably the limit because of memory/speed issues, needs to be tested further
output_list = []
for chunk in pd.read_csv('diffusion_db_unaltered.csv', chunksize=chunk_size):
    chunk['contains_violent'] = chunk['prompt'].str.contains(
        regex,
        na=False # basic substring check, might need to backtrack to v1 substring check
)
violent_rows = chunk[chunk['contains_violent']] #specifically only saving flagged rows to try and optimize, might change later
output_list.append(violent_rows)
# memory cleanup
del chunk
del violent_rows
if output_list: # combining results, may skip this step later
    final_df = pd.concat(output_list, ignore_index=True)
else:
    print("yea there aint no violence here gang")

In [7]:
testing = final_df.sample(10)
print(testing[["prompt", "contains_violent"]])


                                                  prompt  contains_violent
10654  portrait of a cute fat girl with a gun in a ra...              True
11155  leonardo davinci hugging an ai robot professio...              True
3540   photo of a black kitchen, red walls, white flo...              True
6493   kanade tachibana from angel beats | | fine det...              True
6084   award winning photo of some melancholy as anto...              True
9768   Schematic drawing of a gun for angels, concept...              True
5722   maximalist vibrant sculpture of godly street v...              True
8911                               walter white in gta 5              True
4115       attack on titan in the style of cowboy bebop               True
3919              mileena from mortal kombat 3 d render               True
